# seq2seq: Data Preparation

Quarks To Cosmos with AI Virtual Conference: July 12-16, 2021, Carnegie Mellon University

## Contributors

Abdulhakim Alnuqaydan, Ali Kadhim, Sergei Gleyzer, Harrison Prosper

## Hackathon Contributors

Andrew Roberts, Jessica Howard, Samuel Hori, Arvind Balasubramanian, Xiaosheng Zhao, Michael Andrews

July 2021

## Description

Use an encoder/decoder model built using LSTMs to map symbolic mathematical expressions $f(x)$ to their Taylor series expansions to ${\cal O}(x^5)$.

We've heavily borrowed from Charon Guo's excellent tutorial at:

https://charon.me/posts/pytorch/pytorch_seq2seq_1/

### Data Preparation

This notebook performs the following tasks:
  1. Read the sequence pairs from __data/seq2seq_data.txt__.
  1. Exclude
     1. sequences with complex numbers and with Taylor series expansions longer than 1000 characters.
     1. trivial source expressions.
  1. Write the filtered sequences to __data/seq2seq_data_count.txt__, where count is either 10,000 or 60,000 sequences.
  1. Write out __seq2sequtil.py__.
  1. Read filtered data and delimit source (i.e, input) and target (i.e., output) sequences with a tab and newline at the start and end of each sequence, respectively.

In [1]:
import re
import sympy as sp
import numpy as np
import torch

# symbolic symbols
from sympy import exp, \
    cos, sin, tan, \
    cosh, sinh, tanh, ln, log, E
x = sp.Symbol('x')

from IPython.display import display
    
# enable pretty printing of equations
sp.init_printing(use_latex='mathjax')

In [2]:
# Reproducing a subtle bug!
data = [['a(4*x)','b(x)'], ['1(exp)','2'], ['3(x)','4']]
non_trivial = re.compile(r'(a|3)'\
                         '[(].*\bx\b')
# eliminate expressions that do not involve x, exp, cos etc.
data = filter(lambda d: len(non_trivial.findall(d[0])) > 0, data)
data = list(data)
data

[]

### Filter sequences

In [3]:
of_order    = re.compile(' [+] O[(]x[*][*]5.*[)]')
# Please note that breaking a raw string does not propagate its 
# "rawness" across the break :(
non_trivial = re.compile(r'(exp|cos|sin|tan|ln|log|cosh|sinh|tanh)'\
                         r'[(].*\bx\b')
add_count   = re.compile('_data')

def filterData(inpfile='data/seq2seq_data.txt',
               num_seq=60000,  # number of filtered sequence pairs
               min_len=5,      # minimum length of a sequence
               max_len=1000):  # maximum length of a sequence
    
    data = open(inpfile).readlines()
    
    # eliminate expressions involving complex numbers
    data = filter(lambda d: d.find('I') < 0, data)
    data = list(data)

    # strip away O(...) (of order..)
    data = [of_order.sub('', d) for d in data]
 
    # split pairs at tab
    data = [ d.split('\t') for d in data ]
    #print(data[:5])
    
    # keep source expressions that involve exp, cos, etc.
    # that is, eliminate trivial expressions.
    data = filter(lambda d: len(non_trivial.findall(d[0])) > 0, data)
    data = list(data)
 
    # keep expressions that are >= min_len characters long
    data = filter(lambda d: 
                  (len(d[0]) >= min_len) and (len(d[1]) >= min_len),
                  data)
    data = list(data)
                  
    # keep expressions that are <= max_len characters long
    data = filter(lambda d: 
                  (len(d[0]) <= max_len) and (len(d[1]) <= max_len), 
                  data)
    data = list(data)
    
    N = min(num_seq, len(data))
    #print(len(data))
    outfile = add_count.sub('_data_%d' % N, inpfile)
    print('output file:', outfile)
    
    data = ['\t'.join(d) for d in data]
    open(outfile, 'w').writelines(data[:N])
    
filterData(num_seq=10000)
filterData(num_seq=60000)

output file: data/seq2seq_data_10000.txt
output file: data/seq2seq_data_60000.txt


### Map sequences to lists of indices

  1. Split data into a train, validation, and test set.
  1. Create a token (i.e., a character) to index map from training data.
  1. Map sequences to arrays of indices.
  1. Implement custom DataLoader.

In [4]:
%%writefile seq2sequtil.py
import numpy as np
import torch
from IPython.display import display

# symbolic symbols
from sympy import Symbol, exp, \
    cos, sin, tan, \
    cosh, sinh, tanh, ln, log, E
x = Symbol('x')

class Seq2SeqDataPreparer:
    '''
    This class maps the source (i.e., input) and target (i.e, output) 
    sequences of characters into sequences of indices. The source data 
    are split into x_train, x_valid, and x_test sets and similarly for 
    the target data.
    
    Create a data preparer using
    
    dd = Seq2SeqDataPreparer(X, Y, fractions)
    
    where,

      fractions:    a 2-tuple containing the three-way split of data.
                    e.g.: (50/60, 55/60) means split the data as follows
                    (50000, 5000, 5000)
    '''
    def __init__(self, X, Y,
                 fractions=[50/60, 55/60]): 
        
        self.fractions = fractions
        
        # Get maximum sequence length for input expressions
        self.x_max_seq_len =  max([len(z) for z in X])
        
        # Get maximum sequence length for target expressions
        self.y_max_seq_len =  max([len(z) for z in Y])
        
        # get length of splits into train, valid, test
        N = int(len(X)*fractions[0])
        M = int(len(X)*fractions[1])
        
        # Create token to index map for source sequences
        t = self.token_tofrom_index(X[:N])
        self.x_token2index, self.x_index2token = t
        
        # Create token to index map for target sequences
        t = self.token_tofrom_index(Y[:N])
        self.y_token2index,self.y_index2token = t
        
        # Structure data into a list of blocks, where each block
        # comprises a tuple (x_data, y_data) whose elements have
        #   x_data.shape: (x_seq_len, batch_size)
        #   y_data.shape: (y_seq_len, batch_size)
        #
        # The sequence and batch sizes can vary from block to block.
        
        self.train_data, self.n_train = self.code_data(X[:N], Y[:N])         
        self.valid_data, self.n_valid = self.code_data(X[N:M],Y[N:M])
        self.test_data,  self.n_test  = self.code_data(X[M:], Y[M:])

    def __del__(self):
        pass
    
    def __len__(self):
        n  = 0
        n += self.n_train
        n += self.n_valid
        n += self.n_test
        return n
    
    def __str__(self):
        s  = ''
        s += 'number of seq-pairs (train): %8d\n' % self.n_train
        s += 'number of seq-pairs (valid): %8d\n' % self.n_valid
        s += 'number of seq-pairs (test):  %8d\n' % self.n_test
        s += '\n'
        s += 'number of source tokens:     %8d\n' % \
        len(self.x_token2index)
        s += 'max source sequence length:  %8d\n' % \
        self.x_max_seq_len
        
        try:
            s += '\n'
            s += 'number of target tokens:     %8d\n' % \
            len(self.y_token2index)
            s += 'max target sequence length:  %8d' % \
            self.y_max_seq_len
        except:
            pass

        return s
         
    def num_tokens(self, which='source'):
        if which[0] in ['s', 'i']:
            return len(self.x_token2index)
        else:
            return len(self.y_token2index)
    
    def max_seq_len(self, which='source'):
        if which[0] in ['s', 'i']:
            return self.x_max_seq_len
        else:
            return self.y_max_seq_len
        
    def decode(self, indices):
        # map list of indices to a list of tokens
        return ''.join([self.y_index2token[i] for i in indices])

    def token_tofrom_index(self, expressions):
        chars = set()
        chars.add(' ')  # for padding
        chars.add('?')  # for unknown characters
        for expression in expressions:
            for char in expression:
                chars.add(char)
        chars = sorted(list(chars))
        
        char2index = dict([(char, i) for i, char in enumerate(chars)])
        index2char = dict([(i, char) for i, char in enumerate(chars)])
        return (char2index, index2char)
       
    def get_block_indices(self, X, Y):
        # X, and Y are just arrays of strings.
        #
        # 1. Following Michael Andrews' suggestion double sort 
        #    expressions, first with targets then sources. But, also
        #    note the ordinal values "i" of the expressions in X, Y.
        sizes = [(len(a), len(b), i) 
                 for i, (a, b) in enumerate(zip(Y, X))]
        sizes.sort()
  
        # 2. Find ordinal values (indices) of all expression pairs 
        #    for which the sources are the same length and the
        #    targets are the same length. In general, the sources and
        #    targets differ in length.
     
        block_indices = []
        n, m, i  = sizes[0] # n, m, i = len(target), len(source), index
        previous = (n, m)
        indices  = [i] # cache index of first expression
        
        for n, m, i in sizes[1:]: # skip first expression
            
            size = (n, m)
            
            if size == previous:
                indices.append(i) # cache index of expression
            else:
                # found a new boundary, so save previous 
                # set of indices...
                block_indices.append(indices)
                
                # ...and start a new list of indices
                indices = [i]

            previous = size
            
        # cache expression indices of last block
        block_indices.append(indices)
        
        return block_indices
    
    
    def make_block(self, expressions, indices, token2index, unknown):
        
        # batch size of current block
        batch_size = len(indices)
        
        # By construction, all expressions of a block have 
        # the same length, so can use the length of first expression
        seq_len = len(expressions[indices[0]])
        
        # Create an empty block of correct shape and size
        data    = np.zeros((seq_len, batch_size), dtype='long')
        #print('seq_len, batch_size: (%d, %d)' % (seq_len, batch_size))
        
        # loop over expressions for current block
        # m: ordinal value of expression in current block
        # k: ordinal value of expression in original list of expressions
        # n: ordinal value of character in a given expression
        
        for m, k in enumerate(indices):
            
            expr = expressions[k]
            
            #print('%5d expr[%d] | %s |' % (m, k, expr[1:-1]))
            
            # copy coded characters to 2D arrays
        
            for n, char in enumerate(expr):
                #print('\t\t(n, m): (%d, %d)' % (n, m))
                try:
                    data[n, m] = token2index[char]
                except:
                    data[n, m] = unknown
                    
        return data
    
    def code_data(self, X, Y):
        # Implement Arvind's idea
        
        # X, Y consist of delimited strings: 
        #   \tab<characters\newline
        
        # loop over sequence pairs and convert them to sequences
        # of integers using the two token2index maps
      
        x_space   = self.x_token2index[' ']
        x_unknown = self.x_token2index['?']
        
        y_space   = self.y_token2index[' ']
        y_unknown = self.y_token2index['?']
 
        # 1. Get blocks containing sequences of the same length.
        
        block_indices = self.get_block_indices(X, Y)
        
        # 2. Loop over the indices associated with each block of coded
        #    sequences. The indices are the ordinal values of the
        #    sequence pairs X and Y.
        
        blocks = []
        n_data = 0
       
        for indices in block_indices:

            x_data = self.make_block(X, indices, 
                                     self.x_token2index, x_unknown)
 
            y_data = self.make_block(Y, indices, 
                                     self.y_token2index, y_unknown)

            blocks.append((x_data, y_data))
            n = len(indices)
            n_data += n
        
        assert n_data == len(X)
        
        return blocks, n_data
    
class Seq2SeqDataLoader:
    '''
    dataloader = Seq2seqDataLoader(dataset, device, sample=True)    
    '''
    def __init__(self, dataset, device, sample=True):
        self.dataset = dataset
        self.device  = device
        self.sample  = sample  
        self.init()

    def __iter__(self):
        return self
    
    def __next__(self):
        
        # increment iteration counter
        self.count += 1
        
        if self.count <= self.max_count:
            
            # 1. randomly pick a block or return blocks in order.
            if self.sample:
                k = np.random.randint(len(self.dataset))
            else:
                k = self.count-1 # must subtract one!
            
            # 2. create tensors directly on the device of interest
            X = torch.tensor(self.dataset[k][0], 
                             device=self.device)
            
            Y = torch.tensor(self.dataset[k][1], 
                             device=self.device)
        
            # shape of X and Y: (seq_len, batch_size)
            return X, Y
        else:
            self.count = 0
            raise StopIteration
            
    def init(self, max_count=0, sample=True):
        n_data = len(self.dataset)
        self.max_count = n_data if max_count < 1 else min(max_count, 
                                                          n_data)
        self.sample= sample
        self.count = 0
        
# Delimit each sequence in filtered sequences
# The start of sequence (SOS) and end of sequence (EOS) 
# tokens are "\t" and "\n", respectively.

def loadData(inpfile):
    # format of data:
    # input expression<tab>target expression<newline>
    data = [a.split('\t') for a in open(inpfile).readlines()]
    
    X, Y = [], []
    for i, (x, y) in enumerate(data):
        X.append('\t%s\n' % x)
        # get rid of spaces in target sequence
        y = ''.join(y.split())
        Y.append('\t%s\n' % y)
        
    print('Example source:')
    print(X[-1])
    pprint(X[-1])
    print('Example target:')
    print(Y[-1])
    pprint(Y[-1])

    return (X, Y)

def pprint(expr):
    display(eval(expr))

Overwriting seq2sequtil.py


#### Display a few sequence pairs

In [5]:
import seq2sequtil as sq
import importlib
importlib.reload(sq)
inputs, targets = sq.loadData('data/seq2seq_data_10000.txt')

Example source:
	(4*x**2-1)*cos(2*x+9)/cosh(7*x**3/7)/(8*x-8)*exp(-5*x**2+1)



                   2             
⎛   2    ⎞  1 - 5⋅x              
⎝4⋅x  - 1⎠⋅ℯ        ⋅cos(2⋅x + 9)
─────────────────────────────────
                      ⎛ 3⎞       
        (8⋅x - 8)⋅cosh⎝x ⎠       

Example target:
	E*cos(9)/8+x*(E*cos(9)/8-E*sin(9)/4)+x**2*(-E*sin(9)/4-5*E*cos(9)/4)+x**3*(13*E*sin(9)/6-5*E*cos(9)/4)+x**4*(247*E*cos(9)/48+13*E*sin(9)/6)



 4 ⎛247⋅ℯ⋅cos(9)   13⋅ℯ⋅sin(9)⎞    3 ⎛13⋅ℯ⋅sin(9)   5⋅ℯ⋅cos(9)⎞    2 ⎛  ℯ⋅sin(
x ⋅⎜──────────── + ───────────⎟ + x ⋅⎜─────────── - ──────────⎟ + x ⋅⎜- ──────
   ⎝     48             6     ⎠      ⎝     6            4     ⎠      ⎝     4  

9)   5⋅ℯ⋅cos(9)⎞     ⎛ℯ⋅cos(9)   ℯ⋅sin(9)⎞   ℯ⋅cos(9)
── - ──────────⎟ + x⋅⎜──────── - ────────⎟ + ────────
         4     ⎠     ⎝   8          4    ⎠      8    

### Check data preparer

In [6]:
fractions=[8/10, 9/10]
db = sq.Seq2SeqDataPreparer(inputs, targets, fractions)
print(db)

number of seq-pairs (train):     8000
number of seq-pairs (valid):     1000
number of seq-pairs (test):      1000

number of source tokens:           31
max source sequence length:        81

number of target tokens:           35
max target sequence length:       923


### Check data loader 

In [7]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = sq.Seq2SeqDataLoader(db.train_data, device)

n = 0
print('%5s\t%-20s\t%-20s' % ('block', 'X.shape', 'Y.shape'))
for i, (X, Y) in enumerate(train_loader):
    if i % 1000 == 0:
        print('%5d\t%-20s\t%-20s' % (i, X.shape, Y.shape))
    n += 1
print("\nnumber of blocks: %d" % n)

block	X.shape             	Y.shape             
    0	torch.Size([41, 2]) 	torch.Size([133, 2])
 1000	torch.Size([52, 1]) 	torch.Size([244, 1])
 2000	torch.Size([25, 1]) 	torch.Size([70, 1]) 
 3000	torch.Size([49, 1]) 	torch.Size([293, 1])
 4000	torch.Size([32, 2]) 	torch.Size([90, 2]) 

number of blocks: 4685
